# DES CUDA Application

## Usage

```
./des_cuda <encrypt|decrypt> <input_file> <output_file>
```

Where:
- `encrypt|decrypt`: Mode of operation ('encrypt' to encrypt the input file, 'decrypt' to decrypt it)
- `input_file`: Path to the file to be encrypted/decrypted
- `output_file`: Path where the encrypted/decrypted output will be saved

---
**BUT to make it easier i have created a funtion called compiler_and_run_kernel this function takes only name of the file inside code folder to run , mode ,inputfileName**

Cpu Implementation

In [4]:
compile_and_run_kernel("Cpu_DES", "encrypt", "plaintext0")

#compile_and_run_kernel("Cpu_DES", "decrypt", "plaintext0_encrypted")

Current working directory: e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des
Cpu_DES.cu
tmpxft_0000085c_00000000-10_Cpu_DES.cudafe1.cpp
   Creating library e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\code\bin\Cpu_DES.lib and object e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\code\bin\Cpu_DES.exp
Encrypting 230 blocks...
Time taken: 2.00 milliseconds
Encryption complete. Output written to e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\Input_Files/plaintext0_encrypted.txt
Encrypted file saved to e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\Input_Files/plaintext0_encrypted.txt


'e:\\02_Learn\\01_University\\Senior-1 Spring\\Current\\Parallel Computing\\Labs\\Des\\Input_Files/plaintext0_encrypted.txt'

### **DES Encryption and Decryption**

The following examples demonstrate the DES encryption and decryption functionality.

In [ ]:
# Example 1: Encrypt a plaintext file
compile_and_run_kernel("kernel_DES", "encrypt", "plaintext0", analytics=True)

# Example 2: Decrypt the encrypted file
compile_and_run_kernel("kernel_DES", "decrypt", "plaintext0_encrypted", analytics=False)
# streams + ctr code bock + key generation parallel 

Current working directory: e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des
kernel_DES.cu
tmpxft_000019f0_00000000-10_kernel_DES.cudafe1.cpp
   Creating library e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\code\bin\kernel_DES.lib and object e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\code\bin\kernel_DES.exp
Generating 'C:\Users\basim\AppData\Local\Temp\nsys-report-7842.qdstrm'

[1/6] [0%                          ] profile_plaintext0_encrypt.nsys-rep
[1/6] [0%                          ] profile_plaintext0_encrypt.nsys-rep
[1/6] [==========49%               ] profile_plaintext0_encrypt.nsys-rep
[1/6] [===========50%              ] profile_plaintext0_encrypt.nsys-rep
[1/6] [============55%             ] profile_plaintext0_encrypt.nsys-rep
[1/6] [========================100%] profile_plaintext0_encrypt.nsys-rep
[1/6] [========================100%] profile_plaintext0_encrypt.nsys-rep

[2/6] [0%  

kernel_DES.cu
tmpxft_000054fc_00000000-10_kernel_DES.cudafe1.cpp
   Creating library e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\code\bin\kernel_DES.lib and object e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\code\bin\kernel_DES.exp
Decrypting 230 blocks with 256 threads in 1 cuda blocks...
Decryption complete. Output written to e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\Input_Files/plaintext0_encrypted_decrypted.txt
Decrypted file saved to e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\Input_Files/plaintext0_encrypted_decrypted.txt


'e:\\02_Learn\\01_University\\Senior-1 Spring\\Current\\Parallel Computing\\Labs\\Des\\Input_Files/plaintext0_encrypted_decrypted.txt'

### **Performance Analysis**

Run with analytics to profile the encryption and decryption performance.

In [2]:
# Profile encryption performance on a larger file
compile_and_run_kernel("DES", "encrypt", "large_plaintext", analytics=True)

# Profile decryption performance
compile_and_run_kernel("DES", "decrypt", "large_plaintext_encrypted", analytics=True)

Current working directory: e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des
DES.cu


c1xx: fatal error C1083: Cannot open source file: 'e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\code\DES.cu': No such file or directory


Analytics data saved to e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\analytics_Bin\profile_large_plaintext_encrypt
Encrypted file saved to e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\Input_Files/large_plaintext_encrypted.txt
Current working directory: e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des


Executable not found in current directory or standard search paths
c1xx: fatal error C1083: Cannot open source file: 'e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\code\DES.cu': No such file or directory


DES.cu
Analytics data saved to e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\analytics_Bin\profile_large_plaintext_encrypted_decrypt
Decrypted file saved to e:\02_Learn\01_University\Senior-1 Spring\Current\Parallel Computing\Labs\Des\Input_Files/large_plaintext_encrypted_decrypted.txt


Executable not found in current directory or standard search paths


'e:\\02_Learn\\01_University\\Senior-1 Spring\\Current\\Parallel Computing\\Labs\\Des\\Input_Files/large_plaintext_encrypted_decrypted.txt'

***function to Compile and Run any file in folder code***

In [1]:
def compile_and_run_kernel(name, mode, input_file_name, output_file_name=None, analytics=False):
    import os
    
    # Get current working directory
    cwd = os.getcwd()
    print(f"Current working directory: {cwd}")
    # Create paths
    kernel_src = os.path.join(cwd, f"code/{name}.cu")
    kernel_exe = os.path.join(cwd, f"code/bin/{name}.exe")
    
    # Ensure input and output file paths are correct
    input_file = os.path.join(cwd, f"Input_Files/{input_file_name}.txt")
    if output_file_name is None:
        if mode == "encrypt":
            output_file_name = f"{input_file_name}_encrypted"
        else:  # decrypt mode
            output_file_name = f"{input_file_name}_decrypted"
    output_file = os.path.join(cwd, f"Input_Files/{output_file_name}.txt")
    
    # Create bin directory if it doesn't exist
    os.makedirs(os.path.dirname(kernel_exe), exist_ok=True)
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    # Compile
    !nvcc "{kernel_src}" -o "{kernel_exe}"
    
    # Run with analytics if requested, otherwise run normally
    if analytics:
        # Create analytics_Bin directory if it doesn't exist
        analytics_dir = os.path.join(cwd, "analytics_Bin")
        os.makedirs(analytics_dir, exist_ok=True)
        
        # Set profile output path inside analytics_Bin folder
        profile_output = os.path.join(analytics_dir, f"profile_{input_file_name}_{mode}")
        
        # Run with nsys profiling
        !nsys profile --sample=none --trace=cuda --force-overwrite=true --stats=true --output="{profile_output}" "{kernel_exe}" "{mode}" "{input_file}" "{output_file}"
        print(f"Analytics data saved to {profile_output}")
    else:
        # Run normally
        !"{kernel_exe}" "{mode}" "{input_file}" "{output_file}"
    
    print(f"{mode.capitalize()}ed file saved to {output_file}")
    return output_file

# Example usage:
# output = compile_and_run_kernel("DES", "encrypt", "plaintext", analytics=True)